In [1]:
!conda install -c conda-forge spotipy -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\priya\anaconda3

  added / updated specs:
    - spotipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    redis-py-5.0.1             |     pyhd8ed1ab_0         167 KB  conda-forge
    spotipy-2.23.0             |     pyhd8ed1ab_0          30 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         198 KB

The following NEW packages will be INSTALLED:

  redis-py           conda-forge/noarch::redis-py-5.0.1-pyhd8ed1ab_0 
  spotipy            conda-forge/noarch::spotipy-2.23.0-pyhd8ed1ab_0 

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2023.08.22~ --> conda-forge::ca-certificates-2023.11.17-h56e8100_0 

The following packages will be SUPERSEDED by a higher-priority channel:



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




In [2]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [6]:
# Loading not_hot dataframe

sample_not_hot_songs = pd.read_csv(r"C:\Users\priya\Documents\IRONHACK\Week_6\Day_1\Afternoon\lab-not-hot-songs\songs.csv")

sample_not_hot_songs

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Jonas Brothers,Sucker,181026,False,2019,79,0.842,0.734,1,-5.065,0,0.0588,0.0427,0.000000,0.1060,0.952,137.958,pop
1996,Taylor Swift,Cruel Summer,178426,False,2019,78,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994,pop
1997,Blanco Brown,The Git Up,200593,False,2019,69,0.847,0.678,9,-8.635,1,0.1090,0.0669,0.000000,0.2740,0.811,97.984,"hip hop, country"
1998,Sam Smith,Dancing With A Stranger (with Normani),171029,False,2019,75,0.741,0.520,8,-7.513,1,0.0656,0.4500,0.000002,0.2220,0.347,102.998,pop


In [27]:
sample_hot_songs = pd.read_csv("hot_songs.csv")
sample_hot_songs

FileNotFoundError: [Errno 2] No such file or directory: 'hot_songs.csv'

In [19]:
#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID,
                                                           client_secret=Client_Secret))

In [20]:
# load of datasets 

hot_songs = pd.read_csv(r"C:\Users\priya\Documents\IRONHACK\Week_6\Day_1\Afternoon\lab-not-hot-songs\billboard100.csv")

not_hot_songs = pd.read_csv(r"C:\Users\priya\Documents\IRONHACK\Week_6\Day_1\Afternoon\lab-not-hot-songs\songs.csv")
not_hot_songs = not_hot_songs.drop(columns= ['duration_ms', 'explicit', 'year', 'popularity',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'genre'])
not_hot_songs = not_hot_songs.rename(columns={'song': 'title'})

In [25]:
#chunks of 50 songs
chunks = np.split(hot_songs, 50)

In [21]:
# Building function to search a single song

def search_song(title: str, artist: str, limit: int = 5):
    
    '''
    Searches a song in the spotify API based on three parameters and returns the id
    Inputs:
    - title: name of song
    - artist: artist of song
    - limit
    Outputs:
    - results: id
    '''
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret))
    
    search = title + ' ' + artist
    
    response = sp.search(q=search, limit=limit)
    
    results = response['tracks']['items'][0]['id']
    
    return results

In [24]:
# Testing function

search_song('Britney Spears', 'Oops!...I Did It Again')

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))